In [1]:
import os
import topogenesis as tg
import pyvista as pv
import numpy as np
import math as m
import pickle as pk
import resources.RES as res
from sklearn.preprocessing import minmax_scale as sk_minmax
import pygmo as pg
import trimesh as tm

In [2]:
# import lattice
env_lat_path = os.path.relpath("../data/testcase_lattice.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

# voxel dimension
vox_dim = envelope_lattice.unit[0]*envelope_lattice.unit[1]

# plot dimensions
size = envelope_lattice.bounds[1]-envelope_lattice.bounds[0]
plot_area = vox_dim * size[0] * size[1]

# required FSI - USER INPUT
FSI = 3

area_req = FSI * plot_area
voxs_req = int(area_req / (vox_dim))

# number of variables:
num_var = envelope_lattice.flatten().shape[0]

# actual PV interdependencies array:
sol_interd = pk.load(open("../data/T3_SolG1.pk", "rb")) # interdependencies
sol_blocks = pk.load(open("../data/T3_SolU1.pk", "rb")) # context blocks

# actual daylighting obstructing cost array:
sky_interd = pk.load(open("../data/T3_SkyG1.pk", "rb")) # interdependencies
sky_blocks = pk.load(open("../data/T3_SkyU1.pk", "rb")) # context blocks

# actual sky view factor obstructing cost array:
# svf_norm = pk.load(open("../data/T3_svf_norm.pk", "rb")) # sky view blockage of street level from voxels
# svf_interd = pk.load(open("../data/SvFG1.pk", "rb")) # interdependencies
# svf_blocks = pk.load(open("../data/SvFU1.pk", "rb")) # context blocks

dnr = pk.load(open("../data/T2_dnrval.pk", "rb")) # direct normal radiation

In [3]:
# function for reshaping an array to a lattice
def reshape_and_store_to_lattice(values_list, envelope_lattice):
    env_all_vox_id = envelope_lattice.indices.flatten()
    env_all_vox = envelope_lattice.flatten() # envelope inclusion condition: True-False
    env_in_vox_id = env_all_vox_id[env_all_vox] # keep in-envelope voxels (True)

    # initialize array
    values_array = np.full(env_all_vox.shape, 0.0)
    
    # store values for the in-envelope voxels
    values_array[env_in_vox_id] = values_list

    # reshape to lattice shape
    values_array_3d = values_array.reshape(envelope_lattice.shape)

    # convert to lattice
    values_lattice = tg.to_lattice(values_array_3d, envelope_lattice)

    return values_lattice

In [4]:
# transformation matrix
def transform_mat(value):
    mat = np.identity(4)
    mat[:3,-1] = np.array(value)
    return mat

In [5]:
# function for finding total amount of rays that reach the sun for configuration x with possibility for weight
def solarhits(interdependencies, contextblocks, x, radiation=None):
    
    # mask for finding active voxels that may block the rays
    blockedrays = x[np.newaxis, :, np.newaxis] * interdependencies # all rays that are blocked by the 'active' voxels

    blocked = np.sum(blockedrays, axis=1, dtype='int') # how many times each ray is blocked by another active voxel for each voxel
    context_blocks = x[:, np.newaxis] * contextblocks # whether a ray is blocked by the environment for each voxel
    total_blocks = context_blocks + blocked

    # dnr_reshape = radiation[np.newaxis, :]

    reaches = np.where(total_blocks == 0, 1, 0) # outputs 1 if a ray can reach the voxel, else it outputs 0
    # weighted_hits = dnr_reshape * reaches # optional weighting of the rays

    hits = np.sum(reaches, axis=1) # total number of rays that can reach the current configuration
    # directradiation = np.sum(weighted_hits, axis=1) # total direct normal radiation on the voxels TODO: do we want to use this?

    possiblehits = (np.count_nonzero(x) * interdependencies.shape[2]) - context_blocks.sum()
    
    score = hits.sum()
    return score

In [6]:
# function for finding total amount of rays that reach the sky for configuration x
def skyhits(interdependencies, contextblocks, x):
    
    # mask for finding active voxels that may block the rays
    blockedrays = x[np.newaxis, :, np.newaxis] * interdependencies # all rays that are blocked by the 'active' voxels

    blocked = np.sum(blockedrays, axis=1, dtype='int') # how many times each ray is blocked by another active voxel for each voxel
    context_blocks = x[:, np.newaxis] * contextblocks # whether a ray is blocked by the environment for each voxel
    total_blocks = context_blocks + blocked

    reaches = np.where(total_blocks == 0, 1, 0) # outputs 1 if a ray can reach the voxel, else it outputs 0

    hits = np.sum(reaches, axis=1) # total number of rays that can reach the current configuration

    possiblehits = (np.count_nonzero(x) * interdependencies.shape[2]) - context_blocks.sum()
    
    score = hits.sum()
    return score

In [7]:
# function for finding the compactness of configuration x
def compactness(x, reference_lattice):
    # create the current configuration as a lattice
    curr_envelope = reshape_and_store_to_lattice(x.astype('bool'), reference_lattice)
    # flatten the envelope
    envlp_voxs = curr_envelope.flatten()

    # create stencil
    stencil = tg.create_stencil("von_neumann", 1, 1)
    stencil.set_index([0,0,0], 0)

    # find indices of the neighbours for each voxel 
    neighs = curr_envelope.find_neighbours(stencil)

    # occupation status for the neighbours for each voxel
    neighs_status = envlp_voxs[neighs]

    # for voxels inside the envelope:
    neigh_array = np.array(neighs_status[envlp_voxs.astype("bool")])  

    # when the neighbour's status is False that refers to an outer face
    outer_faces = np.count_nonzero(neigh_array==0)

    # voxel edge length
    l = envelope_lattice.unit[0] # TODO: can we leave this dimension out?

    # calculate total surface area of outer faces
    A_exterior = (l**2)*outer_faces

    # number of in-envelope voxels
    in_voxels = np.count_nonzero(x)

    # calculate total volume inclosed in the envelope
    V = in_voxels * (l**3)

    # edge length of a cube that has the same volume
    l_ref = V**(1/3)

    # calculate ratio
    R_ref = (6*(l_ref**2))/V

    relative_compactness = (A_exterior/V)/R_ref
    return relative_compactness

In [8]:
class pygmo_optimze:

    # Number of dimensions
    def __init__(self, dim):
        self.dim = dim

    # Define objectives    
    def fitness(self, x):
        # total solar hits on the roofs of the configuration (PV potential)
        f1 = -solarhits(sol_interd, sol_blocks, x)

        # total daylighting (sky visibility from building)
        f2 = -skyhits(sky_interd, sky_blocks, x) # daylighting potential of voxels

        # floor space index
        # f3 = -(1 - (abs(voxs_req - sum(x)))/voxs_req) # TODO: this constrains the model too much

        # sky view factor from street level around plot
        # f4 = sum(svf_norm[np.nonzero(x)])

        # relative compactness
        # f5 = -compactness(x, envelope_lattice)

        return [f1, f2]
    
    # Return number of objectives
    def get_nobj(self):
        return 2

    # Return bounds of decision variables
    def get_bounds(self):
        return (np.full((self.dim,),0.),np.full((self.dim,),1.))

    # return number of integer variables (all variables are integer in this case TODO: transparency vectors for smoother shapes/results)
    def get_nix(self):
        return self.dim

    # Return function name
    def get_name(self):
        return "Test function MAX no.1"

In [9]:
# create User Defined Problem
prob = pg.problem(pygmo_optimze(dim = num_var))

In [10]:
# create population
pop = pg.population(prob, size=8)

# the used algorithm
algo = pg.algorithm(pg.ihs(gen=150))

# run optimization
pop = algo.evolve(pop)

# extract results
fits, vectors = pop.get_f(), pop.get_x()

# extract non-dominated fronts
ndf, dl, dc, ndr = pg.fast_non_dominated_sorting(fits)



In [11]:
#plotting
# ax = pg.plot_non_dominated_fronts(pop.get_f()) # plotting the non dominated fronts #TODO: what exactly does this represent?

In [12]:
# the best solutions
best = pg.sort_population_mo(points = pop.get_f())[0] # the best solutions (by population)

In [13]:
print("The best configuration is: \n", pop.get_x()[best], "\n It's fitness is: ", pop.get_f()[best], "\n This is population #", best)

The best configuration is: 
 [1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 1.] 
 It's fitn

In [14]:
pop.get_f() #, pop.get_x()

array([[-30760., -29057.],
       [-30739., -29455.],
       [-30760., -29057.],
       [-30760., -29057.],
       [-30760., -29057.],
       [-30760., -29057.],
       [-29706., -27938.],
       [-29706., -27938.]])

In [15]:
best_lat = envelope_lattice.flatten()*pop.get_x()[3]
best_lat_3D = best_lat.reshape(envelope_lattice.shape)

# convert to lattice
values_lattice = tg.to_lattice(best_lat_3D, envelope_lattice)

In [16]:
# creating the new configuration
# configuration = reshape_and_store_to_lattice(pop.get_x()[3], envelope_lattice)
configuration = values_lattice

In [17]:
# visualize configuration 
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
configuration.fast_vis(p,True,True,color='pink',opacity=1)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(69.43029329077571, 69.43029329077571, 89.43029329077571),
 (0.0, 0.0, 20.0),
 (0.0, 0.0, 1.0)]

In [18]:
# voxel cuboid meshes
vox_cuboids = [tm.creation.box(configuration.unit, transform=transform_mat(ct)) for ct in configuration.centroids]

# number of faces per voxel
f_numb = len(vox_cuboids[0].faces)

# combine voxels into one mesh
combined_voxels = tm.util.concatenate(vox_cuboids)

In [19]:
# retrieve skydome
skydome = pk.load(open("../data/T2_skydome.pk", "rb"))

In [20]:
# retrieve context
context_path = os.path.relpath("../input/testcase_context.obj") 
context_mesh = tm.load(context_path)

In [21]:
# combine voxels and context into one mesh
combined_meshes = tm.util.concatenate(combined_voxels, context_mesh)

# shoot towards the skydome points from all of the voxels
DL_ray_ctr = np.tile(skydome, [len(configuration.centroids),1]) # daylighting ray for each centroid
DL_ctr_ray = np.tile(configuration.centroids, [1, len(skydome)]).reshape(-1, 3) # daylighting centroid for each ray

In [22]:
# intersection of rays from voxel centroids to sky patch objects with all voxel faces
face_id, ray_id = combined_meshes.ray.intersects_id(ray_origins=DL_ctr_ray, ray_directions=DL_ray_ctr, multiple_hits=True)

In [23]:
G2, U2 = res.construct_graph(skydome, face_id, ray_id, configuration, f_numb)

In [105]:
obj_score = skyhits(G2,U2,configuration.flatten()) # score of the configuration output according to objective function
opt_score = pop.get_f()[3][1]*-1 # score according to the cost function in the optimization

obj_score, opt_score

(69102, 29057.0)